In [ ]:
from util import get_spec, DotDict, get_generalization, camel_to_snake, snake_to_camel
from os import path, remove
from textwrap import wrap, indent
from networkx import DiGraph, topological_sort, simple_cycles
from re import compile as re_compile

BASE_DIR = filename = '../django_xmi/models/'
FIELD_MAPPINGS = dict(boolean=dict(name='BooleanField', attrs=[]),
                      string=dict(name='CharField', attrs=['max_length=255']))

FIELD_NAME_REPLACEMENTS = {'class': 'klass'}

In [ ]:
uml = get_spec('http://www.omg.org/spec/UML/20131001/UML.xmi')

In [ ]:
sysml = get_spec('http://www.omg.org/spec/SysML/20150709/SysML.xmi')

# Parse elements into a `DotDict`

In [ ]:
%pdb on 

In [ ]:
ascii_fix.sub()

In [ ]:
elements = DotDict({})
profiles = {'UML': uml.XMI.Package.packagedElement,
            'SysML': sysml.XMI.Profile.packagedElement}
ignore = re_compile('^[aeAE]_')
ascii_fix = re_compile(r'[^\x00-\x7F]+')

for profile_name, profile in profiles.items():
    for package_name, package in profile.items():
        for elem_name, element in package.packagedElement.items():
            element.update({'__package__': package_name,
                            '__profile__': profile_name,
                            '__ignore__': bool(ignore.match(elem_name)),
                            '__modelclass__': get_generalization(element) or 'models.Model',
                            '__docstring__': element.get('ownedComment', {}).get('body', ''),
                            '__is_abstract__': element.get('isAbstract', False)})
            element.__docstring__ = ascii_fix.sub("'", element.__docstring__)
            for attr_name in element.get('ownedAttribute', {}).keys():
                attr = element['ownedAttribute'][attr_name]
                if 'ownedComment' in attr:
                    attr['help_text'] = ascii_fix.sub("'", attr['ownedComment'].get('body', ''))
            elements[camel_to_snake(element.name)] = element

In [ ]:
dependency_graph = DiGraph()
BASE_TYPES = ('element',)

for elem_name, element in elements.items():
    if element.__ignore__:
        continue

    if elem_name in BASE_TYPES:
        element.__modelclass__ = 'models.Model'

    if ',' in element.__modelclass__:
        for dependency in element.__modelclass__.split(', '):
            dependency_graph.add_edge(elem_name, camel_to_snake(dependency))
    else:
        dependency_graph.add_edge(elem_name, camel_to_snake(element.__modelclass__))

sorted_elements = list(topological_sort(dependency_graph))[:-1]
sorted_elements.reverse()

# Declare Helper Functions

In [ ]:
def write_literals(literals, models, element_name):
    choices = []
    i = -1
    for term, literal in literals.items():
        i += 1
        models.write('    {code} = {i}\n'.format(code=term.upper(), i=i))
        choices.append("({code}, '{term}'),\n".format(code=term.upper(), term=term))
    
    if i >= 0:
        models.write('    CHOICES = (\n')
        models.write('        ' + '        '.join(choices))
        models.write('    )\n\n')

        field_str = '    {element_name} = models.IntegerField(choices=CHOICES, default={default})\n'
        models.write(field_str.format(element_name=element_name,default=term.upper()))
        return True
    else:
        return False

In [ ]:
def write_attributes(attributes, models):
    """Write the ownedAttributes to the Django models file."""
    
    # TODO: this should probably be somewhere else
    if all(subattr in attributes for subattr in ('id', 'name', 'type')):
        element['ownedAttribute'] = DotDict({})
        element['ownedAttribute'][attributes.name] = DotDict(attributes)
        attributes = element.get('ownedAttribute', {})
        
    found_attributes = False

    for attr in attributes.values():
        subattrs = []

        if attr.name in FIELD_NAME_REPLACEMENTS:
            attr.name = FIELD_NAME_REPLACEMENTS[attr.name]

        if isinstance(attr.type, str):
            attr.__field__ = 'ForeignKey'
            attr.__other__ = attr.type
        elif isinstance(attr.type, dict):
            if 'idref' in attr.type:
                attr.__field__ = 'ForeignKey'
                attr.__other__ = attr.type['idref'].split('_')[-1]
            elif 'href' in attr.type:
                href = attr.type['href'].split('#')[-1].lower()
                if href in FIELD_MAPPINGS:
                    attr.__field__ = FIELD_MAPPINGS[href]['name']
                    subattrs += FIELD_MAPPINGS[href]['attrs']
                else:
                    attr.__field__ = 'ForeignKey'
                    attr.__other__ = attr.type['href'].split('#')[-1]

        if attr.get('__other__', None):
            if attr.__other__ == element.name:
                attr.__other__ = 'self'
            subattrs.append("'{}'".format(attr.__other__))

        help_text = attr.get('help_text', None)
        if help_text:
            quotation = '"""' if '"' in help_text else '"'
            subattrs += ['help_text={}{}{}'.format(quotation, help_text, quotation)]

        attr.__subattrs__ = ', '.join(subattrs)

        attr.name = camel_to_snake(attr.name)

        models.write('    {name} = models.{__field__}({__subattrs__})\n'.format(**attr))
        
        found_attributes = True

    return found_attributes

# Write Django models to files

In [ ]:
loaded = []
for profile in ('uml', 'sysml'):
    filename = path.join(BASE_DIR, "{}.py".format(profile))
    if path.exists(filename):
        remove(filename)
    with open(filename, 'w') as models:
        models.write('from django.db import models\n')
        for other in loaded:
            models.write('from .{} import *\n'.format(other))
    loaded.append(profile)
        
for element_name in sorted_elements:
    if element_name not in elements:
        print('could not find "{}"'.format(element_name))
        continue
    element = elements[element_name]
    
    filename = path.join(BASE_DIR, "{}.py".format(element.__profile__))
    
    with open(filename, 'a') as models:
        models.write('\n\n')
        models.write('class {name}({__modelclass__}):\n'.format(**element))
        if element.__docstring__:
            models.write('    """\n')
            for line in [indent(s, ' ' * 4) for s in wrap('{__docstring__}\n'.format(**element), 80)]:
                models.write(line + '\n')
            models.write('    """\n')
            
        found_literals = write_attributes(element.get('ownedAttribute', {}), models)
        found_attributes = write_literals(element.get('ownedLiteral', {}), models, element_name)
        
        found_meta = False
        if element.__is_abstract__:
            models.write('\n    class Meta:\n        abstract = True\n')
            found_meta = True

        if not any((found_literals, found_attributes, found_meta)):
            models.write('    pass\n')

In [ ]:
elements.packageable_element.ownedAttribute['visibility']

In [ ]:
elements.named_element.ownedAttribute.visibility

In [ ]:
vis = elements.packageable_element.ownedAttribute.get('visibility')

In [ ]:
vis.defaultValue

In [ ]:
elements.visibility_kind.name

In [ ]:
owned = set()
for elem in elements.values():
    owned = owned.union(set([key for key in elem.keys() if 'owned' in key.lower()]))
{k: k.replace('owned', '').lower() + 's' for k in owned if k[:5] == 'owned'}

In [ ]:
{k for k,v in elements.items() if 'ownedRule' in v.keys()}